In [1]:
# Install PyTorch with GPU (CUDA) support
# For CUDA 12.1 (most common). For other versions, see https://pytorch.org
import subprocess
import sys

subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", "torch", "torchvision", "torchaudio", "--index-url", "https://download.pytorch.org/whl/cu121"])
print("PyTorch with CUDA 12.1 installed successfully!")

PyTorch with CUDA 12.1 installed successfully!


In [2]:
# Ensure required packages are installed in the notebook environment
# (This magic installs packages into the active Jupyter kernel)

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split

# 1. Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. Transforms
IMG_SIZE = 512
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

# 3. Data
# Assumes either Dataset/train & Dataset/val folders, or a single Dataset folder with class subfolders
data_dir = 'Dataset'

if os.path.isdir(os.path.join(data_dir, 'train')) and os.path.isdir(os.path.join(data_dir, 'val')):
    image_datasets = {
        'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=data_transforms['train']),
        'val': datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=data_transforms['val'])
    }
else:
    full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
    # assign appropriate transforms
    train_dataset.dataset.transform = data_transforms['train']
    val_dataset.dataset.transform = data_transforms['val']
    image_datasets = {'train': train_dataset, 'val': val_dataset}

# get classes
if hasattr(image_datasets['train'], 'classes'):
    class_names = image_datasets['train'].classes
else:
    class_names = image_datasets['train'].dataset.classes
NUM_CLASSES = len(class_names)
print(f"Classes: {class_names}")

# dataloaders
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=(x=='train'), num_workers=2)
               for x in ['train', 'val']}

# 4. Model
# compatible with different torchvision versions
try:
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
except Exception:
    model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES)
model = model.to(device)

# 5. Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)

# 6. Training and validation helpers
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    epoch_loss = running_loss / total if total > 0 else 0.0
    epoch_acc = correct / total if total > 0 else 0.0
    return epoch_loss, epoch_acc

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    epoch_loss = running_loss / total if total > 0 else 0.0
    epoch_acc = correct / total if total > 0 else 0.0
    return epoch_loss, epoch_acc

# 7. Run training
EPOCHS = 10
for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(model, dataloaders['train'], criterion, optimizer, device)
    val_loss, val_acc = validate(model, dataloaders['val'], criterion, device)
    print(f"Epoch {epoch+1}/{EPOCHS} Train Loss {train_loss:.4f} Acc {train_acc:.4f} | Val Loss {val_loss:.4f} Acc {val_acc:.4f}")

# 8. Save
torch.save(model.state_dict(), 'resnet50_waste.pth')
print("Saved model: resnet50_waste.pth")

Using device: cpu
Classes: ['aluminium', 'paper', 'plastic']
Epoch 1/10 Train Loss 1.0237 Acc 0.4747 | Val Loss 0.9435 Acc 0.3333
Epoch 2/10 Train Loss 0.8125 Acc 0.6707 | Val Loss 0.6497 Acc 0.9000
Epoch 3/10 Train Loss 0.6955 Acc 0.7596 | Val Loss 0.5450 Acc 0.9333
Epoch 4/10 Train Loss 0.6515 Acc 0.7879 | Val Loss 0.4791 Acc 0.9667
Epoch 5/10 Train Loss 0.5745 Acc 0.7919 | Val Loss 0.4227 Acc 0.9667
Epoch 6/10 Train Loss 0.5386 Acc 0.8121 | Val Loss 0.4414 Acc 0.8333
Epoch 7/10 Train Loss 0.5319 Acc 0.7939 | Val Loss 0.4075 Acc 0.8333
Epoch 8/10 Train Loss 0.4725 Acc 0.8566 | Val Loss 0.3485 Acc 0.9333
Epoch 9/10 Train Loss 0.4536 Acc 0.8828 | Val Loss 0.3391 Acc 0.9667
Epoch 10/10 Train Loss 0.4412 Acc 0.8545 | Val Loss 0.2859 Acc 0.9667
Saved model: resnet50_waste.pth
